Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC3641 - Aprendizaje Automático Basado en Grafos <br>
Segundo Semestre 2025<br>


<h1><center> Tarea 3  </center></h1>
        Profesor: Marcelo Mendoza<br>
        Fecha de entrega: 22 de octubre


---

## Indicaciones

Se debe entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas.

**IMPORTANTE**:
- Se asignará puntaje por el código implementado y los comentarios asociados a resultados.
- El notebook debe tener todas las celdas de código ejecutadas.
- Cualquier instancia de copia resultará en un 1.1 como nota de curso.

---

# Integrantes del grupo

* Estudiante 1: Juan Hernandez
* Estudiante 2: Ignacio Vergara
* Estudiante 3: Diego Larraguibel

# Librerías

In [2]:
import dgl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import networkx as nx

# Parte 1: Relational Graph Convolutional Network (R-GCN) (20 puntos)

En esta primera sección se trabaja con el dataset BGSDataset, disponible en dgl.

Ver enlace: https://www.dgl.ai/dgl_docs/generated/dgl.data.BGSDataset.html#dgl.data.BGSDataset

**Observación**

Para trabajar con el código visto en clases deben ajustar la versión de pytorch.

## 1.1 Conceptos básicos (5 puntos)


Responda las siguientes preguntas:

1. Mencione y describa los elementos que diferencian las arquitecturas **GCN** y **R-GCN**.

2. Explique las principales diferencias en el proceso de entrenamiento de arquitecturas basadas en **R-GCN**, considerando las tareas **Clasificación de nodos** y **Predicción de enlaces**

**Observación**

Para la pregunta 2 se recomienda revisar la publicación **Modeling
relational data with graph convolutional networks** (Schlichtkrull et al., 2018). Disponible en: https://arxiv.org/abs/1703.06103


Respuesta:

## 1.2 Análisis descriptivo (5 puntos)

Grafique el grafo y calcule medidas descriptivas para caracterizarlo. Comente sus resultados.

**Observación**

En caso de no poder graficar la red completa, se recomienda trabajar con un subconjunto de 15.000 nodos (demora cerca de 12 minutos). Esto **SOLO** aplica para este punto. **La actividad 1.3 debe considerar todo el grafo.**

Respuesta:

## 1.3 Implementación R-GCN (10 puntos)

Entrene una arquitectura R-GCN para clasificar entidades. Defina el número de épocas de manera que se garantice la convergencia del entrenamiento y justifique su elección de hiperpámetros.

Grafique la función de pérdida y accuracy. Comente sus resultados.

Respuesta:

# Parte 2: Graph Transformer (20 puntos)

En esta segunda sección se debe trabajar con el conjunto de datos **caco2_wang**, contenido en el benchmark de ADMET.

**Observación**

Para cargar el dataset debe seguir el procedimiento visto en clases (8 - Graph-transformer)

## 2.1 Conceptos básicos (5 puntos)

Responda las siguientes preguntas:

1. En el contexto de Graph Transformers, defina el concepto de **sparseness**. ¿Qué implicancias tiene este concepto en las capacidades de cómputo requeridas para trabajar con dichas arquitecturas?

2. Explique el concepto de **positional encoding** en el contexto de los Graph Transformers y compare su formulación con la empleada en el Transformer original (Vaswani et al., 2017).

**Observación**

Se recomienda revisar la publicación **Attention Is All You Need** (Vaswani et al., 2017). Disponible en: https://papers.neurips.cc/paper/7181-attention-is-all-you-need.pdf

Respuesta:

## 2.2 Implementación Graph Transformer (15 puntos)

Proponga cuatro modelos para la tarea de regresión asociada al dataset **caco2_wang**, utilizando la arquitectura **Graph Transformer**. Para ello, haga modificaciones tanto en el número de cabezales como en el número de capas. Justifique sus decisiones.

Compare sus resultados con el Leaderboard disponible en: https://tdcommons.ai/benchmark/admet_group/01caco2/

**Observación**

* Defina el número de épocas de manera que se garantice la convergencia del entrenamiento.
* Trabaje con los conjuntos ya definidos de train, val y test.

Respuesta:

# Parte 3: Graphormer (20 puntos)

En esta tercera sección se debe trabajar con el dataset **PubMed**.

Este conjunto de datos está disponibles en pytorch-geometric. Ver enlace: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.Planetoid.html#torch_geometric.datasets.Planetoid

A continuación, se presenta el código para cargar el dataset.

In [3]:
from torch_geometric.datasets import Planetoid

In [4]:
dataset = Planetoid('./data/PubMed', 'PubMed') #definir root y el dataset que desea descargar
data = dataset[0]

Processing...
Done!


## 3.1 Conceptos básicos (5 puntos)

Responda las siguientes preguntas:
1. ¿Que limitaciones del **Graph Attention Network** (GAT) intenta superar el Graphormer?
2. Describa las principales diferencias entre **Graphormer** y el **Graph Transformer original**.
3. Explique el concepto de **pooling** en el contexto de grafos. ¿En qué se diferencian **DiffPool** de **MinCutPool**?

Respuesta:

## 3.2 DiffPool (5 puntos)

En esta sección se entrenará un modelo Graphormer utilizando el método de pooling jerárquico DiffPool. Para ello, siga los siguientes pasos:
1. Entrene un Graphormer con pooling jerárquico `DiffPool` con 300 épocas.
2. Por cada 20 épocas, reporte los siguientes indicadores:
    - Pérdida de clasificación
    - Regularización
    - Accuracy del test
    - NMI y ARI
    - Promedio tamaño de clusters ± desviación estandar
3. Entrene t-SNE sobre los nodos en el *embedding* final y sobre los centroides, y grafique los resultados. Finalmente, comente sus resultados.

Respuesta:

## 3.3 MinCutPool (5 puntos)

En esta sección se entrenará un modelo Graphormer utilizando el método de pooling jerárquico MinCut. Para ello, siga los siguientes pasos:
1. Entrene un Graphormer con pooling jerárquico `MinCutPool` con 300 épocas.
2. Por cada 20 épocas, reporte los siguientes indicadores:
    - Pérdida de clasificación
    - Regularización
    - Accuracy del test
    - NMI y ARI
    - Promedio tamaño de clusters ± desviación estandar
3. Entrene t-SNE sobre los nodos en el *embedding* final y sobre los centroides, y grafique los resultados. Finalmente, comente sus resultados.

Respuesta:

## 3.4 Conclusiones: DiffPool vs MinCutPool (5 puntos)

* ¿Qué modelo obtuvo mejor accuracy, NMI o ARI?

* ¿Qué diferencias se observaron en la convergencia de las pérdidas entre DiffPool y MinCutPool?

* ¿Hubo una diferencia clara en la capacidad de los modelos para formar clusters coherentes?

* ¿Cuál método mostró una mejor separación de clases en los gráficos t-SNE?

* ¿Cuál de los métodos recomendarías para este tipo de tarea y por qué?

Respuesta: